

> Klaudia Kromołowska - kromolowska@student.agh.edu.pl 

> Jędrzej Kosior - jkosior@student.agh.edu.pl 


In [60]:
# przygotowanie srodowiska
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist

## [deskew]
SZ=28
affine_flags = cv2.WARP_INVERSE_MAP|cv2.INTER_LINEAR

def deskew(img):
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11']/m['mu02']
    M = np.float32([[1, skew, -0.5*SZ*skew], [0, 1, 0]])
    img = cv2.warpAffine(img,M,(SZ, SZ),flags=affine_flags)
    return img
## [deskew]

def showOpencvImage(image, isGray=False):
    fig = plt.figure(figsize=(6, 6))
    plt.imshow(image, cmap = 'gray')
    plt.show()

def openCVHOG(im):
    winSize = (20,20)
    blockSize = (10,10)
    blockStride = (5,5)
    cellSize = (10,10)
    nbins = 9
    derivAperture = 1
    winSigma = -1.
    histogramNormType = 0
    L2HysThreshold = 0.2
    gammaCorrection = 1
    nlevels = 64
    signedGradients = True

    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels, signedGradients)
    descriptor = np.ravel(hog.compute(im))
    
    return descriptor

**Pobranie, przygotowanie i przekształcenie danych**

In [61]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print("Number of images in test set: ", len(test_images))
print("Number of images in train set: ", len(train_images))

train_deskewed = np.asarray([deskew(im) for im in train_images])
test_deskewed = np.asarray([deskew(im) for im in test_images])

#bez wyrownania
train_nd_hogdata = [openCVHOG(im) for im in train_images]
train_nd_hogdata = np.float32(train_nd_hogdata).reshape(-1,81)
test_nd_hogdata = [openCVHOG(im) for im in test_images]
test_nd_hogdata = np.float32(test_nd_hogdata).reshape(-1,81)

# z wyrownaniem
train_d_hogdata = [openCVHOG(im) for im in train_deskewed]
train_d_hogdata = np.float32(train_d_hogdata).reshape(-1,81)
test_d_hogdata = [openCVHOG(im) for im in test_deskewed]
test_d_hogdata = np.float32(test_d_hogdata).reshape(-1,81)

#SIECI NEURONOWE:
 
#bez wyrownania
neural_train_nd = np.array(train_nd_hogdata).reshape((60000, 81))
neural_train_nd = neural_train_nd.astype('float32') / 255

neural_test_nd = np.array(test_nd_hogdata).reshape((10000, 81))
neural_test_nd = neural_test_nd.astype('float32') / 255

#z wyrowananiem
neural_train_d = np.array(train_d_hogdata).reshape((60000, 81))
neural_train_d = neural_train_d.astype('float32') / 255

neural_test_d = np.array(test_d_hogdata).reshape((10000,81 ))
neural_test_d = neural_test_d.astype('float32') / 255


encoded_train_labels = to_categorical(train_labels)
encoded_test_labels = to_categorical(test_labels)

Number of images in test set:  10000
Number of images in train set:  60000


#**TRENING KLASYCZNYCH KLASYFIKATORÓW I SIECI DLA DANYCH BEZ WYRÓWNANIA**

In [62]:
# SVM

from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

parameters = {'kernel':('linear', 'poly', 'rbf', 'sigmoid'), 'C':[0.1, 1, 2, 5]}
model = GridSearchCV(svm.SVC(), parameters, cv=5, scoring='accuracy', return_train_score=True)
#model = svm.SVC(C=15.5,gamma=0.7)
model = model.fit(train_nd_hogdata[0:600,:],train_labels[0:600])
pred_labels = model.predict(test_nd_hogdata[600:1000,:])

mask = pred_labels==test_labels[600:1000]

correct = np.count_nonzero(mask)
print("Correct: ", correct*100.0/pred_labels.size)

cm = confusion_matrix(test_labels[600:1000], pred_labels)
print(cm)
print(classification_report(test_labels[600:1000], pred_labels))

Correct:  89.25
[[31  0  0  0  0  1  0  0  0  0]
 [ 0 53  0  0  0  0  0  0  0  0]
 [ 0  1 44  0  0  0  0  5  1  1]
 [ 0  0  2 41  0  2  0  0  0  0]
 [ 0  0  0  0 42  0  0  0  0  1]
 [ 0  1  0  0  0 27  0  0  1  2]
 [ 0  0  0  0  0  0 35  0  0  0]
 [ 0  0  5  4  0  1  0 31  0  1]
 [ 1  0  0  1  0  2  1  3 24  5]
 [ 0  0  1  0  0  0  0  0  0 29]]
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        32
           1       0.96      1.00      0.98        53
           2       0.85      0.85      0.85        52
           3       0.89      0.91      0.90        45
           4       1.00      0.98      0.99        43
           5       0.82      0.87      0.84        31
           6       0.97      1.00      0.99        35
           7       0.79      0.74      0.77        42
           8       0.92      0.65      0.76        37
           9       0.74      0.97      0.84        30

    accuracy                           0.89       400
   

In [63]:
# RANDOOM FOREST CLASSIFIER

from sklearn.ensemble import RandomForestClassifier
parameters = {'n_estimators':[50, 80,150], 'max_depth': [2,40,80, None]}
rfc = GridSearchCV(RandomForestClassifier(), parameters, cv=5, scoring='accuracy')
rfc = rfc.fit(train_nd_hogdata[0:600,:],train_labels[0:600])

pred_labels = rfc.predict(test_nd_hogdata[600:1000,:])
mask = pred_labels==test_labels[600:1000]

correct = np.count_nonzero(mask)
print("Correct: ", correct*100.0/pred_labels.size)

cm = confusion_matrix(test_labels[600:1000], pred_labels)
print(cm)
print(classification_report(test_labels[600:1000], pred_labels))


Correct:  89.0
[[30  0  0  0  1  1  0  0  0  0]
 [ 0 51  0  0  0  1  0  1  0  0]
 [ 0  1 45  0  0  0  0  4  0  2]
 [ 0  0  0 44  0  1  0  0  0  0]
 [ 0  0  0  0 39  1  2  0  0  1]
 [ 0  1  0  0  0 29  0  1  0  0]
 [ 1  0  0  0  0  1 33  0  0  0]
 [ 1  0  3  6  0  2  0 30  0  0]
 [ 1  0  1  0  0  3  1  0 26  5]
 [ 0  0  1  0  0  0  0  0  0 29]]
              precision    recall  f1-score   support

           0       0.91      0.94      0.92        32
           1       0.96      0.96      0.96        53
           2       0.90      0.87      0.88        52
           3       0.88      0.98      0.93        45
           4       0.97      0.91      0.94        43
           5       0.74      0.94      0.83        31
           6       0.92      0.94      0.93        35
           7       0.83      0.71      0.77        42
           8       1.00      0.70      0.83        37
           9       0.78      0.97      0.87        30

    accuracy                           0.89       400
   m

In [64]:
#NEURAL NETWORK
from keras.datasets import mnist
from keras import models
from keras import layers
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def makeModel():
  network = models.Sequential()
  network.add(layers.Dense(512, activation='relu', input_shape=(81,)))
  network.add(layers.Dense(10, activation='softmax'))
  network.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
  return network

model = KerasClassifier(makeModel, verbose = 0)


parameters = dict(batch_size=[5,20,50], epochs = [15])
network = GridSearchCV(estimator=model, param_grid=parameters, n_jobs=-1, cv=3)
network.fit(neural_train_nd, encoded_train_labels)

pred= network.predict(neural_test_nd[600:1000,])
test_acc=accuracy_score(test_labels[600:1000], pred)
print("Correct: ", test_acc*100)
cm = confusion_matrix(test_labels[600:1000], pred)
print(cm)
print(classification_report(test_labels[600:1000], pred))



Correct:  91.25
[[29  0  0  0  0  1  1  0  0  1]
 [ 0 52  0  0  1  0  0  0  0  0]
 [ 0  1 48  0  0  0  0  2  1  0]
 [ 0  0  2 42  0  1  0  0  0  0]
 [ 0  0  0  0 41  0  1  0  0  1]
 [ 0  0  0  1  0 27  0  0  1  2]
 [ 1  0  0  0  0  0 34  0  0  0]
 [ 1  0  7  3  0  1  0 29  0  1]
 [ 0  0  1  0  0  2  0  0 33  1]
 [ 0  0  0  0  0  0  0  0  0 30]]
              precision    recall  f1-score   support

           0       0.94      0.91      0.92        32
           1       0.98      0.98      0.98        53
           2       0.83      0.92      0.87        52
           3       0.91      0.93      0.92        45
           4       0.98      0.95      0.96        43
           5       0.84      0.87      0.86        31
           6       0.94      0.97      0.96        35
           7       0.94      0.69      0.79        42
           8       0.94      0.89      0.92        37
           9       0.83      1.00      0.91        30

    accuracy                           0.91       400
   

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


#**TRENING KLASYCZNYCH KLASYFIKATORÓW I SIECI DLA DANYCH Z WYRÓWNANIEM**

In [65]:
# SVM

from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix

parameters = {'kernel':('linear', 'poly', 'rbf', 'sigmoid'), 'C':[0.1, 1, 2, 5]}
model = GridSearchCV(svm.SVC(), parameters, cv=5, scoring='accuracy', return_train_score=True)
#model = svm.SVC(C=15.5,gamma=0.7)
model = model.fit(train_d_hogdata[0:600,:],train_labels[0:600])
pred_labels = model.predict(test_d_hogdata[600:1000,:])

mask = pred_labels==test_labels[600:1000]

correct = np.count_nonzero(mask)
print("Correct: ", correct*100.0/pred_labels.size)

cm = confusion_matrix(test_labels[600:1000], pred_labels)
print(cm)
print(classification_report(test_labels[600:1000], pred_labels))

Correct:  91.0
[[32  0  0  0  0  0  0  0  0  0]
 [ 0 52  0  0  0  0  0  1  0  0]
 [ 0  1 48  0  0  0  0  2  1  0]
 [ 0  0  4 36  0  2  0  0  2  1]
 [ 0  0  0  0 42  0  0  1  0  0]
 [ 0  0  0  0  0 29  0  1  1  0]
 [ 0  0  0  0  0  0 35  0  0  0]
 [ 1  0  4  4  0  1  0 32  0  0]
 [ 0  0  1  0  0  2  0  0 31  3]
 [ 0  0  1  0  0  0  0  2  0 27]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.98        32
           1       0.98      0.98      0.98        53
           2       0.83      0.92      0.87        52
           3       0.90      0.80      0.85        45
           4       1.00      0.98      0.99        43
           5       0.85      0.94      0.89        31
           6       1.00      1.00      1.00        35
           7       0.82      0.76      0.79        42
           8       0.89      0.84      0.86        37
           9       0.87      0.90      0.89        30

    accuracy                           0.91       400
   m

In [66]:
# RANDOOM FOREST CLASSIFIER

from sklearn.ensemble import RandomForestClassifier

parameters = {'n_estimators':[50, 80,150], 'max_depth': [2,40,80, None]}
rfc = GridSearchCV(RandomForestClassifier(), parameters, cv=5, scoring='accuracy')
#rfc =  RandomForestClassifier(max_depth=15, n_estimators=100, max_features=60)
rfc = rfc.fit(train_d_hogdata[0:600,:],train_labels[0:600])

pred_labels = rfc.predict(test_d_hogdata[600:1000,:])
mask = pred_labels==test_labels[600:1000]

correct = np.count_nonzero(mask)
print("Correct: ", correct*100.0/pred_labels.size)

cm = confusion_matrix(test_labels[600:1000], pred_labels)
print(cm)
print(classification_report(test_labels[600:1000], pred_labels))


Correct:  88.5
[[30  0  0  0  0  1  1  0  0  0]
 [ 0 52  0  0  0  0  0  1  0  0]
 [ 0  1 47  0  0  1  0  2  0  1]
 [ 0  0  0 40  0  1  0  1  3  0]
 [ 0  0  0  0 42  0  0  0  0  1]
 [ 1  0  0  1  0 28  0  0  1  0]
 [ 1  0  0  0  0  0 34  0  0  0]
 [ 1  0  6  4  1  0  0 30  0  0]
 [ 1  0  1  1  0  2  0  0 24  8]
 [ 0  0  1  0  1  0  0  1  0 27]]
              precision    recall  f1-score   support

           0       0.88      0.94      0.91        32
           1       0.98      0.98      0.98        53
           2       0.85      0.90      0.88        52
           3       0.87      0.89      0.88        45
           4       0.95      0.98      0.97        43
           5       0.85      0.90      0.88        31
           6       0.97      0.97      0.97        35
           7       0.86      0.71      0.78        42
           8       0.86      0.65      0.74        37
           9       0.73      0.90      0.81        30

    accuracy                           0.89       400
   m

In [67]:
#NEURAL NETWORK
from keras.datasets import mnist
from keras import models
from keras import layers
from sklearn.metrics import classification_report, confusion_matrix

model = KerasClassifier(makeModel, verbose = 0)


parameters = dict(batch_size=[5,20,50], epochs = [15])
network = GridSearchCV(estimator=model, param_grid=parameters, n_jobs=-1, cv=3)
network.fit(neural_train_d, encoded_train_labels)

pred= network.predict(neural_test_d[600:1000,])
test_acc=accuracy_score(test_labels[600:1000], pred)
print("Correct: ", test_acc*100)
cm = confusion_matrix(test_labels[600:1000], pred)
print(cm)
print(classification_report(test_labels[600:1000], pred))



Correct:  95.5
[[32  0  0  0  0  0  0  0  0  0]
 [ 0 52  0  0  1  0  0  0  0  0]
 [ 0  1 47  0  0  0  0  2  2  0]
 [ 0  0  0 44  0  1  0  0  0  0]
 [ 0  0  0  0 42  0  0  1  0  0]
 [ 0  0  0  0  0 30  0  1  0  0]
 [ 0  0  0  0  0  0 35  0  0  0]
 [ 0  0  0  5  0  0  0 36  0  1]
 [ 0  0  0  0  0  2  0  0 34  1]
 [ 0  0  0  0  0  0  0  0  0 30]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        32
           1       0.98      0.98      0.98        53
           2       1.00      0.90      0.95        52
           3       0.90      0.98      0.94        45
           4       0.98      0.98      0.98        43
           5       0.91      0.97      0.94        31
           6       1.00      1.00      1.00        35
           7       0.90      0.86      0.88        42
           8       0.94      0.92      0.93        37
           9       0.94      1.00      0.97        30

    accuracy                           0.95       400
   m

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


# **TRENING KLASYCZNYCH KLASYFIKATORÓW DLA DANYCH SUROWYCH WGRANYCH NA WEJŚCIE KLASYFIKATORA**

In [68]:
#przygotowanie danych

train_reshaped = np.float32(train_deskewed).reshape(-1,28*28)
test_reshaped = np.float32(test_deskewed).reshape(-1,28*28)

neural_train_reshaped = np.array(train_deskewed).reshape((60000, 28*28))
neural_train_reshaped = neural_train_reshaped.astype('float32') / 255

neural_test_reshaped = np.array(test_deskewed).reshape((10000, 28*28))
neural_test_reshaped = neural_test_reshaped.astype('float32') / 255


In [72]:
# SVM

from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix


parameters = {'kernel':('linear', 'poly', 'rbf', 'sigmoid'), 'C':[0.1, 1, 2, 5]}
model = GridSearchCV(svm.SVC(), parameters, cv=5, scoring='accuracy', return_train_score=True)
#model = svm.SVC(C=15.5,gamma=0.7)
model = model.fit(train_reshaped[0:600,:],train_labels[0:600])
pred_labels = model.predict(test_reshaped[600:1000,:])

mask = pred_labels==test_labels[600:1000]

correct = np.count_nonzero(mask)
print("Correct: ", correct*100.0/pred_labels.size)

cm = confusion_matrix(test_labels[600:1000], pred_labels)
print(cm)
print(classification_report(test_labels[600:1000], pred_labels))

Correct:  90.75
[[30  0  1  0  0  1  0  0  0  0]
 [ 0 53  0  0  0  0  0  0  0  0]
 [ 1  0 48  0  0  0  0  1  2  0]
 [ 0  0  1 36  0  6  0  0  1  1]
 [ 0  0  0  0 38  0  1  0  0  4]
 [ 1  0  0  0  0 29  0  1  0  0]
 [ 1  0  2  0  0  0 32  0  0  0]
 [ 0  0  1  0  2  0  0 38  0  1]
 [ 1  0  0  1  0  4  0  0 31  0]
 [ 1  0  0  0  0  0  0  1  0 28]]
              precision    recall  f1-score   support

           0       0.86      0.94      0.90        32
           1       1.00      1.00      1.00        53
           2       0.91      0.92      0.91        52
           3       0.97      0.80      0.88        45
           4       0.95      0.88      0.92        43
           5       0.72      0.94      0.82        31
           6       0.97      0.91      0.94        35
           7       0.93      0.90      0.92        42
           8       0.91      0.84      0.87        37
           9       0.82      0.93      0.87        30

    accuracy                           0.91       400
   

In [70]:
# RANDOOM FOREST CLASSIFIER

from sklearn.ensemble import RandomForestClassifier

parameters = {'n_estimators':[50, 80,150], 'max_depth': [2,40,80, None]}
rfc = GridSearchCV(RandomForestClassifier(), parameters, cv=5, scoring='accuracy')
#rfc =  RandomForestClassifier(max_depth=15, n_estimators=100, max_features=60)
rfc = rfc.fit(train_reshaped[0:600,:],train_labels[0:600])

pred_labels = rfc.predict(test_reshaped[600:1000,:])
mask = pred_labels==test_labels[600:1000]

correct = np.count_nonzero(mask)
print("Correct: ", correct*100.0/pred_labels.size)

cm = confusion_matrix(test_labels[600:1000], pred_labels)
print(cm)
print(classification_report(test_labels[600:1000], pred_labels))


Correct:  87.25
[[30  0  0  0  0  1  1  0  0  0]
 [ 0 53  0  0  0  0  0  0  0  0]
 [ 0  0 47  0  0  0  1  1  3  0]
 [ 0  0  1 38  0  3  0  0  1  2]
 [ 0  0  0  0 33  1  1  0  0  8]
 [ 1  0  0  0  1 26  0  2  0  1]
 [ 1  0  1  0  1  0 32  0  0  0]
 [ 0  2  4  0  1  0  0 33  0  2]
 [ 3  0  0  0  0  2  0  0 31  1]
 [ 0  0  0  0  2  0  0  2  0 26]]
              precision    recall  f1-score   support

           0       0.86      0.94      0.90        32
           1       0.96      1.00      0.98        53
           2       0.89      0.90      0.90        52
           3       1.00      0.84      0.92        45
           4       0.87      0.77      0.81        43
           5       0.79      0.84      0.81        31
           6       0.91      0.91      0.91        35
           7       0.87      0.79      0.82        42
           8       0.89      0.84      0.86        37
           9       0.65      0.87      0.74        30

    accuracy                           0.87       400
   

In [71]:
#NEURAL NETWORK
from keras.datasets import mnist
from keras import models
from keras import layers
from sklearn.metrics import classification_report, confusion_matrix

def makeModelN():
  network = models.Sequential()
  network.add(layers.Dense(512, activation='relu', input_shape=(28*28,)))
  network.add(layers.Dense(10, activation='softmax'))
  network.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
  return network


model = KerasClassifier(makeModelN, verbose = 0)


parameters = dict(batch_size=[5], epochs = [15])
network = GridSearchCV(estimator=model, param_grid=parameters, n_jobs=-1, cv=3)
network.fit(neural_train_reshaped[0:600,], encoded_train_labels[0:600])

pred= network.predict(neural_test_reshaped[600:1000,])
test_acc=accuracy_score(test_labels[600:1000], pred)
print("Correct: ", test_acc*100)
cm = confusion_matrix(test_labels[600:1000], pred)
print(cm)
print(classification_report(test_labels[600:1000], pred))



Correct:  89.75
[[31  0  0  1  0  0  0  0  0  0]
 [ 0 52  0  0  0  0  1  0  0  0]
 [ 1  1 47  1  0  0  0  1  1  0]
 [ 0  2  1 36  0  4  0  0  0  2]
 [ 0  0  0  0 39  0  1  0  0  3]
 [ 1  0  0  0  1 25  1  3  0  0]
 [ 2  0  0  0  1  1 31  0  0  0]
 [ 0  0  0  0  1  0  0 40  0  1]
 [ 2  0  0  0  0  0  0  1 32  2]
 [ 1  0  0  0  0  0  0  3  0 26]]
              precision    recall  f1-score   support

           0       0.82      0.97      0.89        32
           1       0.95      0.98      0.96        53
           2       0.98      0.90      0.94        52
           3       0.95      0.80      0.87        45
           4       0.93      0.91      0.92        43
           5       0.83      0.81      0.82        31
           6       0.91      0.89      0.90        35
           7       0.83      0.95      0.89        42
           8       0.97      0.86      0.91        37
           9       0.76      0.87      0.81        30

    accuracy                           0.90       400
   

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
